In [1]:
import os
import sys
sys.path.append("../Processor")
import time
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision

from data_processor import SpeechDigitsDataset, BinningHistogram, Pad
from models import SNN, SpikingConv2DLayer, ReadoutLayer, SurrogateHeaviside, SpikingDenseLayer

In [2]:
dtype = torch.float
device = torch.device("cpu")

In [4]:
train_dataset_raw = SpeechDigitsDataset(data_root="", transform = None, mode="train", nb_digits=1)
print(train_dataset_raw[0][0].shape)
print(train_dataset_raw[1][0].shape)
max_end_time = train_dataset_raw.get_max_end_time()
print(max_end_time / 0.005)
size = 375

(898, 2)
(1026, 2)
370.8927869796753


In [26]:
data_root = ""
binning_method = "time"
T_l = 0.005
batch_size = 32


binning = BinningHistogram(binning_method=binning_method, T_l=T_l)
pad = Pad(size)
transform = torchvision.transforms.Compose([binning,
                                 pad])

train_dataset = SpeechDigitsDataset(data_root, transform = transform, mode="train", nb_digits=1)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

In [29]:
print(train_dataset[0][0].shape)
print(train_dataset[1][0].shape)

(375, 64)
(375, 64)


In [28]:
X_batch, _ = next(iter(train_dataloader))
X_batch = X_batch.to(device, dtype)
print(X_batch.shape)

torch.Size([32, 375, 64])


In [51]:
spike_fn = SurrogateHeaviside.apply

w_init_std = 0.15
w_init_mean = 0.

layers = []
in_channels = 1
out_channels = 64
kernel_size = (4,3)
dilation = (1,1)
input_shape = 64
output_shape = input_shape # padding mode is "same"
layers.append(SpikingConv2DLayer(input_shape, output_shape,
                 in_channels, out_channels, kernel_size, dilation,
                 spike_fn, w_init_mean=w_init_mean, w_init_std=w_init_std, recurrent=False,
                               lateral_connections=True))


in_channels = out_channels
out_channels = 64
kernel_size = (4,3)
dilation = (4,3)
input_shape = output_shape
output_shape = input_shape # padding mode is "same"
layers.append(SpikingConv2DLayer(input_shape, output_shape,
                 in_channels, out_channels, kernel_size, dilation,
                 spike_fn, w_init_mean=w_init_mean, w_init_std=w_init_std, recurrent=False,
                              lateral_connections=True))

in_channels = out_channels
out_channels = 64
kernel_size = (4,3)
dilation = (16,9)
input_shape = output_shape
output_shape = input_shape # padding mode is "same"
layers.append(SpikingConv2DLayer(input_shape, output_shape,
                 in_channels, out_channels, kernel_size, dilation,
                 spike_fn, w_init_mean=w_init_mean, w_init_std=w_init_std, recurrent=False,
                               lateral_connections=True, flatten_output=True))

# previous layer output has been flattened
input_shape = output_shape*out_channels
output_shape = 11
time_reduction="mean" #mean or max
layers.append(ReadoutLayer(input_shape, output_shape,
                 w_init_mean=w_init_mean, w_init_std=w_init_std, time_reduction=time_reduction))

snn = SNN(layers).to(device, dtype)

X_batch, _ = next(iter(train_dataloader))
X_batch = X_batch.to(device, dtype)
#you need to add a channel dimension
X_batch = X_batch.unsqueeze(1)
snn(X_batch)

for i,l in enumerate(snn.layers):
    if isinstance(l, SpikingDenseLayer) or isinstance(l, SpikingConv2DLayer):
        print("Layer {}: average number of spikes={:.4f}".format(i,l.spk_rec_hist.mean()))

Layer 0: average number of spikes=0.0189
Layer 1: average number of spikes=0.0153
Layer 2: average number of spikes=0.0220
